In [1]:
import numpy as np 
import cv2,joblib
import Sliding as sd
from imutils.object_detection import non_max_suppression
import imutils
from skimage.feature import hog
from skimage import color
from skimage.transform import pyramid_gaussian
import json
import os

size = (64,128)
step_size = (9,9)
downscale = 1.25

coco_results = []

with open('output/Ground Truth.json', 'r') as f:
    coco_data = json.load(f)
image_id_map = {image['file_name']: image['id'] for image in coco_data['images']}
print(image_id_map)

model = joblib.load('models/models.dat')

image_folder = 'images'
for image_filename in os.listdir(image_folder):
    image_path = os.path.join(image_folder, image_filename)
    image = cv2.imread(image_path)

    print(f"Processing {image_filename}...")

    image_id = image_id_map.get(image_filename)
    image = cv2.resize(image,(400,256))

    #List to store the detections
    detections = []
    #The current scale of the image 
    scale = 0


    for im_scaled in pyramid_gaussian(image, downscale = downscale):
        #The list contains detections at the current scale
        if im_scaled.shape[0] < size[1] or im_scaled.shape[1] < size[0]:
            break
        for (x, y, window) in sd.sliding_window(im_scaled, size, step_size):
            if window.shape[0] != size[1] or window.shape[1] != size[0]:
                continue
            window = color.rgb2gray(window)
                
            fd=hog(window, orientations=9,pixels_per_cell=(8,8),visualize=False,cells_per_block=(3,3))
            fd = fd.reshape(1, -1)
            pred = model.predict(fd)
            if pred == 1:
                    
                if model.decision_function(fd) > 0.5:
                    detections.append((int(x * (downscale**scale)), 
                                        int(y * (downscale**scale)), 
                                        model.decision_function(fd), 
                                        int(size[0] * (downscale**scale)),
                                        int(size[1] * (downscale**scale))))
        scale += 1


    clone = image.copy()
    rects = np.array([[x, y, x + w, y + h] for (x, y, _, w, h) in detections])
    sc = [score[0] for (x, y, score, w, h) in detections]
    sc = np.array(sc)
    pick = non_max_suppression(rects, probs=sc, overlapThresh=0.5)

    for (x1, y1, x2, y2), confidence in zip(pick, sc):
        # 绘制矩形框
        cv2.rectangle(clone, (x1, y1), (x2, y2), (0, 255, 0), 2)
        # 在矩形框旁边显示标签和置信度
        cv2.putText(clone, f'Person: {confidence:.2f}', (x1-2, y1-2), 1, 0.75, (121, 12, 34), 1)
        bbox = [int(x1), int(y1), int(x2 - x1), int(y2 - y1)]
        coco_results.append({
            "image_id": int(image_id),
            "category_id": 0,
            "bbox": bbox,
            "score": float(confidence)
        })

    save_folder = 'detectResult/HOG'
    if not os.path.exists(save_folder):
        os.makedirs(save_folder)
    output_filename = f"{save_folder}/{image_filename}"
    cv2.imwrite(output_filename, clone)
    print(f"Detection result saved to {output_filename}")

output_filename = 'output/HOG.json'
with open(output_filename, 'w') as f:
    json.dump(coco_results, f)
print(f"Detection results saved to {output_filename}")


# cv2.imshow('Person Detection',clone)
# cv2.waitKey(0)
# cv2.destroyAllWindows()


{'b71184b9-1.jpg': 0, 'b2dd240c-2.jpg': 1, 'e628b968-3.jpg': 2, 'c1c0fe3c-4.jpg': 3, '16bf9a67-0.jpg': 4, 'ad3956f8-9.jpg': 5, '0dd20243-8.jpg': 6, 'd9a9c2fa-5.jpg': 7, '9a07eb57-6.jpg': 8, '796ec187-7.jpg': 9}


d:\conda_envs\envs\detectron2\lib\site-packages\sklearn\base.py:347: InconsistentVersionWarning: Trying to unpickle estimator LinearSVC from version 0.22.1 when using version 1.3.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


Processing 0dd20243-8.jpg...
Detection result saved to detectResult/HOG/0dd20243-8.jpg
Processing 16bf9a67-0.jpg...
Detection result saved to detectResult/HOG/16bf9a67-0.jpg
Processing 796ec187-7.jpg...
Detection result saved to detectResult/HOG/796ec187-7.jpg
Processing 9a07eb57-6.jpg...
Detection result saved to detectResult/HOG/9a07eb57-6.jpg
Processing ad3956f8-9.jpg...
Detection result saved to detectResult/HOG/ad3956f8-9.jpg
Processing b2dd240c-2.jpg...
Detection result saved to detectResult/HOG/b2dd240c-2.jpg
Processing b71184b9-1.jpg...
Detection result saved to detectResult/HOG/b71184b9-1.jpg
Processing c1c0fe3c-4.jpg...
Detection result saved to detectResult/HOG/c1c0fe3c-4.jpg
Processing d9a9c2fa-5.jpg...
Detection result saved to detectResult/HOG/d9a9c2fa-5.jpg
Processing e628b968-3.jpg...
Detection result saved to detectResult/HOG/e628b968-3.jpg
Detection results saved to output/HOG.json
